# Pytorch Demo for Deep Learning 2021
## Description:
Demo contains code to classify cats and dogs dataset from kaggle. Classifier is built using fully connected layers from pytorch nn module. Code is run on google colab and it works correctly. Students are also suggested to take advantge of colab facility. Functionality of different parts is explained a little below. I have shared the dataset with all students you can access it using this link. but you can only access it using your university email.

train dataset=https://drive.google.com/drive/folders/16YIhnoi1sgYulknym39KtmetHhCxN2N8?usp=sharing

test dataset=https://drive.google.com/drive/folders/1hbC5sWl6Z1WI20u2iX1Yxwzrmzz8RvSp?usp=sharing

First cell below imports required packages.

In [3]:
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models


#### Two cells given below import google drive as a folder to colab environment. If you are using your own computer for this demo ignore these cell. comment them out.

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# drive.mount("/content/drive", force_remount=True)

## Tranforms and Dataloader
#### Code in the cell below loads dataset so that we can train model on it and test our trained model. First two statements define tranforms to transform our dataset. Images are resized to (100*100) and normalized and converted to pytorch tensors. It is not necessary to resize images but it is done for this tutorial to save a lot trianing time. Normalizing datasets have advantages that you guys will learn in future lectures.
#### Image folder is loaded using ImageFolder Object and then it is passed as argument to Dataloader Object to load data at run time.

In [6]:
# !unzip -uq "/content/drive/MyDrive/datasets/testCD-20210402T171330Z-001.zip" -d "/content/drive/MyDrive/datasets/"



In [7]:
# !unzip -uq "/content/drive/MyDrive/datasets/train-20210402T171330Z-001.zip" -d "/content/drive/MyDrive/datasets/"

In [8]:
data_dir = '/content/drive/MyDrive/datasets' #DOWNLOAD DATA FOR THIS

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Loading dataset using ImageFolder method. Pass transforms in here.
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/testCD', transform=test_transforms)

# Here train and test dataloader are created using train_data and test_data objects. Batch size tells how many images are 
# loaded for one iteration. 

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)


## Declaring Neural Network
#### This cell defines our neural network. Our neural network has five layers. First layer is input layer, then next three layers are hidden layers and last layer is output layer. Input to our network is an (100*100*3) sized image. So our input layer has 30000 neurons, next three layers have 5000, 500 and 100 neurons respectively. Then the last layer has only two neurons because our dataset has only two classes. Constructor of class is just initializing these layers.
#### Forward function is compulsery to implement if we are extending nn.Module class. It specify's what happens to the input image. Relu is used as non-linearity in this case. 


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(100 * 100*3, 5000)
        self.fc2 = nn.Linear(5000, 500)
        self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(100, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x,dim=1)

In this cell we create object of Net class and specify learning rate and number of epochs. 

In [10]:
model = Net()
learning_rate=0.0005
epochs=10

# this statement tell our code if there gpu available on our machine or not.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Net(
  (fc1): Linear(in_features=30000, out_features=5000, bias=True)
  (fc2): Linear(in_features=5000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=2, bias=True)
)

In this cell we choose optimizer and loss function. We are using stochastic gradient descent as our optimizer and cross entropy loss as our loss function. You will learning how these algorithms work in class lectures in detail.

In [11]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

## Training the Network
Training neural network is done in multiple steps. After extracting data and target from the train loader, we assign them to the device. Our network takes 1 dimensional input tensor but images are 3 dimensional tensors, so they are converted to 1d tensors using (view) function. torch accumulates gradients of all steps so we have to zero all previous gradients, this is done with zero_grad() function. After this data is passed through network and loss is calculated based on predictions and targets then this loss is passed backward. This is done using loss.backward() function.   

In [12]:
# for epoch in range(epochs):
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = Variable(data), Variable(target)
#         data, target = data.to(device), target.to(device)        
#         data = data.view(-1, 100*100*3)
#         optimizer.zero_grad()
#         net_out = model(data)
#         # print('Targets',target)
#         # print("Prediction",net_out)
#         loss = criterion(net_out, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                     epoch, batch_idx * len(data), len(train_loader.dataset),
#                            100. * batch_idx / len(train_loader), loss.item()))

In [13]:
# this line of code saves the trained model to google drive. You can specify path on local drive.
# torch.save(model.state_dict(), "content/drive/MyDrive/datasets/train/weights.pth")

In [14]:
# this loads the saved model from give directory.
model.load_state_dict(torch.load('/content/drive/MyDrive/datasets/train/weights.pth'))

<All keys matched successfully>

In [15]:
# this code cell calculates accuracy of test data on the trained model.
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)

        data = data.view(-1, 100 * 100*3)
        net_out = model(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).item()
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0105, Accuracy: 1397/2009 (70%)

